In [1]:
import numpy as np 
import pandas as pd 

import os

import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras

import multiprocessing

from tensorflow.keras.layers import Dense, Input, Conv2D, Flatten
from tensorflow.keras.applications import EfficientNetB0, Xception

from tensorflow.keras import Model

# from skimage.io import imread
import cv2

from skimage.transform import resize
import numpy as np
import math

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import matplotlib.pyplot as plt


In [2]:
#train = pd.read_csv("train.csv")


In [3]:
#train.head()

In [4]:
#path = "train_images/"
#train['full_filepath'] = path + train.chain.astype(str) +"/"+ train.image.astype(str)

In [5]:
#train.head()

In [6]:
#train.iloc[0,4]

In [7]:
#train = train[train.chain.isin([0,1,2])]
#train.shape

In [8]:
#n_subsample = 5000
#train = train.sample(n_subsample)

In [9]:
#X_train, X_val, = train_test_split(train, test_size = 0.30,
#    stratify = train['chain'], shuffle = True
#)

In [10]:
#print(X_train.shape)
#print(X_val.shape)

In [11]:
#n_classes = X_train.chain.nunique()

#BATCH_SIZE = 64
#STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE
EPOCHS = 50

IMG_HEIGHT = 299
IMG_WIDTH = 299
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)

In [12]:
'''
class HotelBatchSequence(tf.keras.utils.Sequence):
    
    def __init__(self, x_set, y_set, batch_size,
                 img_size = (224, 224),
                 augment = False):
        """
        `x_set` is list of paths to the images
        `y_set` are the associated classes.

        """
        
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.img_size = img_size
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        """Generate one batch of data"""
        
        first_id = idx * self.batch_size
        last_id =  (idx + 1) * (self.batch_size)
        
        batch_x = self.x[first_id:last_id]
        batch_y = self.y[first_id:last_id]
        
        #Xs = np.array([resize(imread(file_name), self.img_size)
        #      for file_name in batch_x])
        # 
        #ys = np.array(batch_y)
        
        output = np.array([
            resize(cv2.imread(file_name), self.img_size)
                   for file_name in batch_x]), np.array(batch_y)
        
        return output
        
'''

'\nclass HotelBatchSequence(tf.keras.utils.Sequence):\n    \n    def __init__(self, x_set, y_set, batch_size,\n                 img_size = (224, 224),\n                 augment = False):\n        """\n        `x_set` is list of paths to the images\n        `y_set` are the associated classes.\n\n        """\n        \n        self.x = x_set\n        self.y = y_set\n        self.batch_size = batch_size\n        self.img_size = img_size\n    \n    def __len__(self):\n        """Denotes the number of batches per epoch"""\n        return math.ceil(len(self.x) / self.batch_size)\n    \n    def __getitem__(self, idx):\n        """Generate one batch of data"""\n        \n        first_id = idx * self.batch_size\n        last_id =  (idx + 1) * (self.batch_size)\n        \n        batch_x = self.x[first_id:last_id]\n        batch_y = self.y[first_id:last_id]\n        \n        #Xs = np.array([resize(imread(file_name), self.img_size)\n        #      for file_name in batch_x])\n        # \n       

In [13]:
'''
TrainGenerator = HotelBatchSequence(X_train.full_filepath, 
                                    tf.keras.utils.to_categorical(X_train.chain),
                                    BATCH_SIZE)

ValidGenerator = HotelBatchSequence(X_val.full_filepath, 
                                   tf.keras.utils.to_categorical(X_val.chain),
                                   BATCH_SIZE)
                                   '''

'\nTrainGenerator = HotelBatchSequence(X_train.full_filepath, \n                                    tf.keras.utils.to_categorical(X_train.chain),\n                                    BATCH_SIZE)\n\nValidGenerator = HotelBatchSequence(X_val.full_filepath, \n                                   tf.keras.utils.to_categorical(X_val.chain),\n                                   BATCH_SIZE)\n                                   '

In [14]:
batch_size = 10

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'newimages/train',  # this is the target directory
        target_size=(299, 299),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
        'newimages/test',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='sparse')





'''
train_generator = train_datagen.flow_from_directory(
        'data3/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'data3/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
'''

Found 51787 images belonging to 87 classes.
Found 25508 images belonging to 87 classes.


"\ntrain_generator = train_datagen.flow_from_directory(\n        'data3/train',  # this is the target directory\n        target_size=(224, 224),  # all images will be resized to 150x150\n        batch_size=batch_size,\n        class_mode='categorical')\n\ntest_generator = test_datagen.flow_from_directory(\n        'data3/test',\n        target_size=(224, 224),\n        batch_size=batch_size,\n        class_mode='categorical')\n"

In [15]:
#test_batch=TrainGenerator.__getitem__(0)

In [16]:
#test_batch[0][0].shape

In [17]:
#tf.keras.backend.clear_session()

In [18]:
efficientnet = Xception(include_top=True,
                             weights='imagenet',
                             input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
                              



91889664/91884032 [==============================] - 1s 0us/step


In [19]:
flatten1= Flatten()(efficientnet.output)
output = Dense(87, activation='softmax')(flatten1)

In [20]:
model = Model(inputs=efficientnet.input,outputs=output)

In [21]:
tf.__version__

'2.4.1'

In [22]:

#model = efficientnet

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')

In [23]:
'''
class SGDR(tf.keras.callbacks.Callback):
 

    def __init__(self, min_lr=0.0, max_lr=0.05, base_epochs=10, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        return self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs))
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            '''

'\nclass SGDR(tf.keras.callbacks.Callback):\n \n\n    def __init__(self, min_lr=0.0, max_lr=0.05, base_epochs=10, mul_epochs=2):\n        super(SGDR, self).__init__()\n\n        self.min_lr = min_lr\n        self.max_lr = max_lr\n        self.base_epochs = base_epochs\n        self.mul_epochs = mul_epochs\n\n        self.cycles = 0.\n        self.cycle_iterations = 0.\n        self.trn_iterations = 0.\n\n        self._reset()\n\n    def _reset(self, new_min_lr=None, new_max_lr=None,\n               new_base_epochs=None, new_mul_epochs=None):\n        """Resets cycle iterations."""\n        \n        if new_min_lr != None:\n            self.min_lr = new_min_lr\n        if new_max_lr != None:\n            self.max_lr = new_max_lr\n        if new_base_epochs != None:\n            self.base_epochs = new_base_epochs\n        if new_mul_epochs != None:\n            self.mul_epochs = new_mul_epochs\n        self.cycles = 0.\n        self.cycle_iterations = 0.\n        \n    def sgdr(self):\n 

In [25]:
history = model.fit(train_generator,
                 #   steps_per_epoch = STEPS_PER_EPOCH,
                    validation_data = test_generator,
                  #  validation_split=.10,
                    workers = -1,
                    epochs = 10,
                    use_multiprocessing = False,
                    max_queue_size = 10
                    )

Epoch 1/10
 537/5179 [==>...........................] - ETA: 1:06:53 - loss: 4.0804 - accuracy: 0.1035

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/PIL/Image.py:2850: DecompressionBombWarning: Image size (108576768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


5179/5179 [==============================] - 5692s 1s/step - loss: 3.6733 - accuracy: 0.1056 - val_loss: 3.5806 - val_accuracy: 0.1055
Epoch 2/10
5179/5179 [==============================] - 5642s 1s/step - loss: 3.5796 - accuracy: 0.1055 - val_loss: 3.5774 - val_accuracy: 0.1055
Epoch 3/10
5179/5179 [==============================] - 5641s 1s/step - loss: 3.5783 - accuracy: 0.1055 - val_loss: 3.5774 - val_accuracy: 0.1055
Epoch 4/10
5179/5179 [==============================] - 5646s 1s/step - loss: 3.5782 - accuracy: 0.1055 - val_loss: 3.5774 - val_accuracy: 0.1055
Epoch 5/10
5179/5179 [==============================] - 5624s 1s/step - loss: 3.5784 - accuracy: 0.1055 - val_loss: 3.5773 - val_accuracy: 0.1055
Epoch 6/10
5179/5179 [==============================] - 5604s 1s/step - loss: 3.5783 - accuracy: 0.1055 - val_loss: 3.5774 - val_accuracy: 0.1055
Epoch 7/10
5179/5179 [==============================] - 5651s 1s/step - loss: 3.5784 - accuracy: 0.1055 - val_loss: 3.5774 - val_accura

In [ ]:
model.summary()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.save('eff_net1')

In [ ]:
keras.models.save_model(history, KERAS_MODEL_NAME)